# 08 Semantic Kernel | 08 Map Reduce/Summarizer

## Map Reduce/Summarizer with Semantic Kernel



## Azure Environment

To execute the sample code Azure service specific information like endpoint, api key etc. is needed ([Details and instructions can be found here](../01_DemoEnvironment/01_Environment.ipynb)) 

## Step 1: Create Semantic Kernel instance

An instance of Semantic Kernel can be created using the KernelBuilder object provided by the Semantic Kernel SDK. It acts as the centralized point for all .NET functionality that want to interact with Semantic Kernel functionality or concepts.

It abstracts e.g., models from the OpenAI GPT family and can communicate with Azure OpenAI LLM instances as well as LLMs deployed on OpenAI. In the sample we use models deployed on Azure.

In [ ]:
#r "nuget: Microsoft.SemanticKernel, 1.3.0"
#r "nuget: Microsoft.SemanticKernel.Core, 1.3.0"
#r "nuget: DotNetEnv, 2.5.0"

using System;
using System.IO;
using System.Text;
using System.Text.RegularExpressions;
using System.Text.Json;
using System.Text.Json.Serialization;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Connectors.OpenAI;

using DotNetEnv;
using InteractiveKernel = Microsoft.DotNet.Interactive.Kernel;

#!import Models/Models.cs
#!import Utilities/TextChunker.cs
#!import Templates/Templates.cs

const int ChunkSize = 1000;
const int MaxTokens = 1500;
const double Temperature = 0.3;

static string _configurationFile = @"../01_DemoEnvironment/conf/application.env";
Env.Load(_configurationFile);

string oAiApiKey = Environment.GetEnvironmentVariable("SKIT_AOAI_APIKEY") ?? "SKIT_AOAI_APIKEY not found";
string oAiEndpoint = Environment.GetEnvironmentVariable("SKIT_AOAI_ENDPOINT") ?? "SKIT_AOAI_ENDPOINT not found";
string chatCompletionDeploymentName = Environment.GetEnvironmentVariable("SKIT_CHATCOMPLETION_DEPLOYMENTNAME") ?? "SKIT_CHATCOMPLETION_DEPLOYMENTNAME not found";

#pragma warning disable CS8618,IDE0009,CA1051,CA1050,CA1707,CA2007,VSTHRD111,CS1591,RCS1110,CA5394,SKEXP0001,SKEXP0002,SKEXP0003,SKEXP0004,SKEXP0010,SKEXP0011,SKEXP0012,SKEXP0020,SKEXP0021,SKEXP0022,SKEXP0023,SKEXP0024,SKEXP0025,SKEXP0026,SKEXP0027,SKEXP0028,SKEXP0029,SKEXP0030,SKEXP0031,SKEXP0032,SKEXP0040,SKEXP0041,SKEXP0042,SKEXP0050,SKEXP0051,SKEXP0052,SKEXP0053,SKEXP0054,SKEXP0055,SKEXP0060,SKEXP0061,SKEXP0101,SKEXP0102
var kernel = Kernel.CreateBuilder()
    .AddAzureOpenAIChatCompletion(chatCompletionDeploymentName, oAiEndpoint, oAiApiKey)
    .Build();


#### Expected output

```
Installed Packages
dotenv.net, 2.5.0
Microsoft.SemanticKernel, 1.3.0
Microsoft.SemanticKernel.Core, 1.3.0
```

## Step 2: Read the transcript and break it into chunks

In [ ]:
var text = File.ReadAllText("data/transcript.txt");
var chunks = TextChunker.ChunkText(text, ChunkSize);

## Step 3: For each chunk produce a summary and save the summary

In [ ]:
foreach(var chunk in chunks)
{
    var prompt = SummaryTemplate.Replace("{context}", chunk.Content);
    var chunkSummaryFunction = kernel.CreateFunctionFromPrompt(prompt);
    var result = await kernel.InvokeAsync(chunkSummaryFunction);
    chunk.Summary = result.ToString();
}

## Step 4: Combine all the chunk summaries and produce a full summary for all the chunks

In [ ]:
var augmentedContext = new StringBuilder();

foreach(var chunk in chunks)
{
    augmentedContext.Append(chunk.Summary+"\n\n");    
}

var fullSummaryTemplate = FinalSummaryTemplate.Replace("{context}", augmentedContext.ToString());

// Create a Prompt SK function from the template
var fullSummaryFunction = kernel.CreateFunctionFromPrompt(fullSummaryTemplate,new OpenAIPromptExecutionSettings() { MaxTokens = MaxTokens, Temperature = Temperature, TopP = 1 });

// Run the function
var result = await kernel.InvokeAsync(fullSummaryFunction);
result.ToString()